In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

from dotenv import load_dotenv 
import os 
load_dotenv()

api_key = os.getenv("GEMINI_API_KEY")

In [2]:
model_client = OpenAIChatCompletionClient(
    model="gemini-1.5-flash-8b",
    api_key=api_key)

In [8]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination

dsa_solver = AssistantAgent(
    name = 'Complex_DSA_Solver',
    model_client=model_client,
    description='A DSA solver',
    system_message="You give code in python to solve complex DSA problems. Give under 100 words"
)

code_reviewer = AssistantAgent(
    name = 'CODE_REVEIWER',
    model_client=model_client,
    description='A Code Reviewer',
    system_message="You review the code given by the complex_dsa_solver and make sure it is optimized.Give under 10 word."
)

code_editor = AssistantAgent(
    name = 'CODE_EDITOR',
    model_client=model_client,
    description='A Code editor',
    system_message="You make the code easy to understand and add comments wherever required.Give under 10 words."
)

In [9]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination

termination_condition = TextMentionTermination("TERMINATE") | MaxMessageTermination(1)

team = RoundRobinGroupChat(
    participants=[dsa_solver,code_reviewer,code_editor],
    termination_condition=termination_condition
)

# async def test_team():
#     task = TextMessage(content='Write a simple Hello world code ?',source='User')
#     result = await team.run(task=task)
#     for each_agent_message in result.messages:
#         print(f'{each_agent_message.source} : {each_agent_message.content}' )
#         print('\n \n')

# await test_team()

In [10]:
from autogen_agentchat.ui import Console
await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a short poem about the fall season."))

---------- TextMessage (user) ----------
Write a short poem about the fall season.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a short poem about the fall season.', type='TextMessage')], stop_reason='Maximum number of messages 1 reached, current message count: 1')